In [ ]:
%%writefile app.py
# Creamos el archivo de la APP en el intérprete principal (Python)

#################################################################
# Importamos librerías 
import streamlit as st
import plotly.express as px
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.metrics import confusion_matrix 
import seaborn as sns  





#Personalizamos el dashboard 
st.markdown("""
    <style>
        [data-testid="stSidebar"] {
            background-color: #C7D9DD;
        }
        div[data-baseweb="select"] > div {
            background-color: #EEF1DA; 
        }
    </style>
""", unsafe_allow_html=True)

st.sidebar.markdown("""
    <h2 style='text-align: center; color: #4A4A4A;'> DASHBOARD </h2>
""", unsafe_allow_html=True)




#Cargamos los datos 
@st.cache_resource
def load_data():
    df = pd.read_csv("ValoresAtipicosMilan.csv", index_col='id')
    
    # Seleccion de columnas de tipo categoricas 
    text_df = df.select_dtypes(['object'])
    text_cols = text_df.columns

    return df, text_cols


df, text_cols = load_data()

# Selector de vista 
tipo_vista = st.sidebar.selectbox("Tipo de análisis exploratorio", ["Información","Análisis univariado", "Análisis de correlación", "Análisis logistico"])


#Separador 
st.sidebar.markdown("---")

if tipo_vista == "Información":
    st.header("Milán, Italia")
    

    col1, col2 = st.columns([2, 2])  # 1 parte imagen, 3 partes texto

    with col1:
        st.image("images/milan.jpg", width=400)

    with col2:
       st.markdown("""
        Milán no es solo una ciudad, es un estilo de vida.  
        Entre avenidas elegantes y fachadas históricas, Milán marca el ritmo del diseño, la moda y la creatividad italiana.  
        Desde el imponente Duomo hasta las boutiques de lujo en Via Montenapoleone, cada rincón respira arte, arquitectura y glamour.  
        Es un lugar donde las ideas desfilan como en pasarela y donde cada detalle importa.  
        **Milano, dove tutto è stile.** 
        """)

    st.markdown("---")

    st.subheader("¿Por qué analizar Airbnb en Milán?")
    st.markdown("""
    Milán es uno de los destinos más visitados del norte de Italia, no solo por su arquitectura y moda,  
    sino por ser sede de eventos globales como la **Semana de la Moda**, ferias de diseño y exposiciones de arte. Esto genera una **alta demanda de alojamiento temporal**, ideal para análisis de tendencias en **Airbnb**.
    """)

    st.image("images/semanaDeLaModa.jpeg", width=650)

    st.markdown("---")

   
    st.subheader("Datos generales del dataset")

    col1, col2 = st.columns([2, 2])  

    with col1:
        st.write(f"-Número total de registros: {df.shape[0]}")
        st.write(f"-Número de columnas: {df.shape[1]}")
        st.write(f"-Hosts únicos: {df['host_id'].nunique()}")
        st.write(f"-Vecindarios distintos: {df['neighbourhood'].nunique()}")

        if 'price' in df.columns:
            st.write(f"-Precio promedio por noche: €{df['price'].mean():.2f}")
            st.write(f"-Alojamiento más caro: €{df['price'].max()}")
            st.write(f"-Alojamiento más barato: €{df['price'].min()}")
    
    with col2:
        st.image("images/milan_mapa.jpg", width=250)

    
    st.markdown("### Dataset Completo")
    st.write(df)
    st.write(df.describe())

    st.markdown("---")

 
    if 'latitude' in df.columns and 'longitude' in df.columns:
        st.subheader("Ubicación de alojamientos")
        st.map(df[['latitude', 'longitude']])

    

elif tipo_vista == "Análisis univariado":
    st.header("Análisis univariado")

    categoricas_directas = [
        "host_is_superhost",
        "host_identity_verified",
        "neighbourhood_cleansed",
        "property_type",
        "room_type",
        "instant_bookable",
        "host_response_time",
        "has_availability"
    ]

    # Copia del dataframe para no alterar el original
    df_cat = df.copy()

    
    df_cat["accommodates_cat"] = pd.cut(df_cat["accommodates"],
                                        bins=[0, 1, 2, 4, np.inf],
                                        labels=["1 persona", "2 personas", "3-4 personas", "5 o más"])


    df_cat["bedrooms_cat"] = pd.cut(df_cat["bedrooms"],
                                    bins=[-1, 0, 1, 2, 3, np.inf],
                                    labels=["Sin habitación", "1", "2", "3", "4 o más"])

    df_cat["beds_cat"] = pd.cut(df_cat["beds"],
                                bins=[-1, 0, 1, 2, 3, np.inf],
                                labels=["Sin cama", "1", "2", "3", "4 o más"])


    df_cat["minimum_nights_cat"] = pd.cut(df_cat["minimum_nights"],
                                        bins=[0, 1, 3, 7, np.inf],
                                        labels=["1 noche", "2-3 noches", "4-7 noches", "Más de 7 noches"])

    df_cat["maximum_nights_cat"] = pd.cut(df_cat["maximum_nights"],
                                        bins=[0, 7, 30, 90, np.inf],
                                        labels=["1-7 noches", "8-30 noches", "31-90 noches", "Más de 90 noches"])

    df_cat["availability_365_cat"] = pd.cut(df_cat["availability_365"],
                                            bins=[-1, 0, 30, 90, 180, 365],
                                            labels=["Sin disponibilidad", "1-30 días", "31-90 días", "91-180 días", "181-365 días"])
    
    categoricas_convertidas = [
        "accommodates_cat",
        "bedrooms_cat",
        "beds_cat",
        "minimum_nights_cat",
        "maximum_nights_cat",
        "availability_365_cat"
    ]

    # Concatenar todas las opciones
    opciones_variables = categoricas_directas + categoricas_convertidas


    variable_objetivo = st.sidebar.selectbox("Variable objetivo", opciones_variables)
    tipo_grafica = st.sidebar.selectbox("Tipo de gráfica", ["Gráfica de pastel", "Gráfica de barras"])
    generar = st.sidebar.button("Generar gráfica")
    
    palette =  [
    '#00A8E8',  
    '#90E0EF',  
    '#F4D35E',  
    '#EE964B',  
    '#F95738',  
    '#43AA8B',  
    '#FF6B6B',  
    ]

    # Inicializamos la lista de gráficas en session_state
    if "graficas_generadas" not in st.session_state:
        st.session_state.graficas_generadas = []

    # Si se presiona el botón, agregamos la gráfica a la lista
    if generar:
       # Verificar si ya existe esa combinación
        ya_existe = any(
            g['variable'] == variable_objetivo and g['tipo'] == tipo_grafica 
            for g in st.session_state.graficas_generadas
        )

        if ya_existe:
            st.error(f"¡Ya has generado esta gráfica de {tipo_grafica} para la variable '{variable_objetivo}'!")
        else:
            # Si no existe, la agregas normal
            st.session_state.graficas_generadas.append({
                "variable": variable_objetivo,
                "tipo": tipo_grafica,
                "id": len(st.session_state.graficas_generadas)  # ID único por posición
            })
    # Mostrar todas las gráficas guardadas
    for grafica in st.session_state.graficas_generadas:
        st.subheader(f"Frecuencia de {grafica['variable']}")

        conteo = df_cat[grafica['variable']].value_counts().reset_index()
        conteo.columns = [grafica['variable'], 'Frecuencia']

        # Mostrar gráfica según el tipo
        if grafica['tipo'] == "Gráfica de barras":
            fig = px.bar(conteo,
             x=grafica['variable'],
             y='Frecuencia',
             text='Frecuencia',
             color=grafica['variable'],
             color_discrete_sequence=palette)

        elif grafica['tipo'] == "Gráfica de pastel":
            fig = px.pie(conteo,
             names=grafica['variable'],
             values='Frecuencia',
             title=f"Distribución de {grafica['variable']}",
             color_discrete_sequence=palette)


        # Contenedor con botón eliminar
        with st.container():
            st.plotly_chart(fig, use_container_width=True)
            eliminar = st.button(f"Eliminar gráfica {grafica['variable']}", key=f"del_{grafica['id']}")
            if eliminar:
                st.session_state.graficas_generadas = [
                    g for g in st.session_state.graficas_generadas if g["id"] != grafica["id"]
                ]
                st.rerun() 


    if not st.session_state.get("graficas_generadas"):
        st.markdown("""
           <div style="background-color:#90E0EF; padding:15px; border-radius:10px; color:#003049;">
            <h4>Análisis univariado de frecuencia</h4>
            <p>El análisis univariado de frecuencia nos permite conocer 
            <strong>cuántas veces aparece cada valor o categoría dentro de una variable</strong>. 
            Esto es muy útil para entender la distribución de los datos, identificar valores atípicos o poco comunes, 
            y asegurarnos de que la información esté bien representada antes de continuar con otros análisis.</p>

            <p>Para realizar este tipo de análisis, selecciona la variable objetivo, el tipo de gráfica que deseas generar 
            y haz clic en el botón <strong>"Generar gráfica"</strong>.</p>
            </div>
        """, unsafe_allow_html=True)


elif tipo_vista == "Análisis de correlación":
    st.header("Análisis de Correlación")
    st.sidebar.header("Regresión lineal simple")

    #------------------------------------------------------------------
    df_num = df.copy()
    df_num['host_is_superhost'] = df_num['host_is_superhost'].map({'t': 1, 'f': 0})
    df_num['host_identity_verified'] = df_num['host_identity_verified'].map({'t': 1, 'f': 0})
    df_num['instant_bookable'] = df_num['instant_bookable'].map({'t': 1, 'f': 0})
    df_num['has_availability'] = df_num['has_availability'].map({'t': 1, 'f': 0})



    variables_permitidas = [
    "price", "review_scores_rating", "number_of_reviews", "availability_365",
    "host_response_rate", "host_acceptance_rate", "host_listings_count",
    "host_total_listings_count", "accommodates", "bedrooms", "beds", "bathrooms",
    "availability_30", "availability_90", "review_scores_accuracy", 
    "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication",
    "review_scores_location", "review_scores_value",
    "host_is_superhost", "host_identity_verified", "instant_bookable", "has_availability"  
    ]

    df_num = df_num[variables_permitidas]

    
    variable_dependiente = st.sidebar.selectbox("Variable dependiente", variables_permitidas, key="dependiente_simple")
    variable_independiente = st.sidebar.selectbox(
        "Variable independiente", 
        [col for col in variables_permitidas if col != variable_dependiente], key="independiente_simple"
    )
    generar = st.sidebar.button("Generar gráfica", key="btn_simple")
    st.sidebar.markdown("---")


    st.sidebar.header("Regresión lineal multiple")
    varDependiente = st.sidebar.selectbox("Variable dependiente",  variables_permitidas,  key="dependiente_multiple")
    varIndependiente = st.sidebar.multiselect(label="Variables independientes", options=  variables_permitidas, key="independientes_multiple")
    generarMultiple = st.sidebar.button("Generar gráfica", key="btn_multiple")
    st.sidebar.markdown("---")

    #-----------------------HEADMAP------------------------------------
    mostrar_heatmap = st.sidebar.checkbox("Mostrar mapa de calor", value=True)

    if mostrar_heatmap:
        matriz_corr = df_num.corr()
        fig = px.imshow(
            matriz_corr,
            text_auto=True,
            color_continuous_scale='RdBu_r',
            aspect='auto',
            height=800,
            title="Mapa de calor de correlaciones"
        )
        st.plotly_chart(fig, use_container_width=True)

    # Inicializamos la lista si no existe
    if "graficas_guardadas" not in st.session_state:
        st.session_state["graficas_guardadas"] = []

    if "graficas_guardadas_multiple" not in st.session_state:
        st.session_state["graficas_guardadas_multiple"] = []

    

    #-----------------------Regresión lineal simple------------------------------------
    # Si se genera la gráfica, guardamos solo los datos
    if generar:
        ya_existe = any(
            g['variable_dependiente'] == variable_dependiente and 
            g['variable_independiente'] == variable_independiente 
            for g in st.session_state["graficas_guardadas"]
        )

        if ya_existe:
            st.error(f"¡Ya generaste la gráfica de {variable_dependiente} vs {variable_independiente}! Selecciona otra combinación.")
        else:
            # Si NO existe, entonces sí generas y guardas la gráfica:
            X = df_num[[variable_independiente]].values
            y = df_num[variable_dependiente].values

            model = LinearRegression()
            model.fit(X, y)
            pendiente = model.coef_[0]
            intercepto = model.intercept_
            ecuacion = f"y = {pendiente:.4f}x + {intercepto:.4f}"
            r = np.corrcoef(X.flatten(), y)[0, 1]
            r2 = model.score(X, y)

            st.session_state["graficas_guardadas"].append({
                "variable_independiente": variable_independiente,
                "variable_dependiente": variable_dependiente,
                "ecuacion": ecuacion,
                "r": r,
                "r2": r2,
                "id": len(st.session_state["graficas_guardadas"])  # ID único por gráfica
            })


    if st.session_state["graficas_guardadas"]:
        st.subheader("Regresión Lineal Simple") 
        
    for grafica in st.session_state["graficas_guardadas"]:
        x = df_num[[grafica['variable_independiente']]].values
        y = df_num[grafica['variable_dependiente']].values

        model = LinearRegression()
        model.fit(x, y)
        y_pred = model.predict(x)

        fig = px.scatter(df, 
                         x=grafica['variable_independiente'], 
                         y=grafica['variable_dependiente'],
                         title=f"{grafica['variable_dependiente']} en función de {grafica['variable_independiente']}")

        fig.add_scatter(x=df[grafica['variable_independiente']], y=y_pred,
                        mode='lines', name='Línea de regresión', line=dict(color='red'))

        st.subheader(f"{grafica['variable_dependiente']} vs {grafica['variable_independiente']}")
        i = 0
        st.plotly_chart(fig, use_container_width=True)
        st.write(f"**Ecuación de la recta:** {grafica['ecuacion']}")
        st.write(f"**Coeficiente de correlación (r):** {grafica['r']:.4f}")
        st.write(f"**Coeficiente de determinación (R²):** {grafica['r2']:.4f}")

        # Botón para eliminar gráfica específica
        if st.button(f"Eliminar gráfica", key=f"del_{grafica['id']}"):
            st.session_state["graficas_guardadas"] = [
                g for g in st.session_state["graficas_guardadas"] if g["id"] != grafica["id"]
            ]
            st.rerun()
            break
    #-----------------------Regresión lineal multiple-----------------------------------
    
    if generarMultiple and varIndependiente:
        ya_existe = any(
            g['var_dependiente'] == varDependiente and 
            set(g['var_independiente']) == set(varIndependiente)
            for g in st.session_state["graficas_guardadas_multiple"]
        )

        if ya_existe:
            st.error(f"¡Ya generaste esta regresión múltiple para la variable dependiente '{varDependiente}' con esas variables independientes!")
        else:
            
            X_multi = df_num[varIndependiente]
            y_multi = df_num[varDependiente]

            model_multi = LinearRegression()
            model_multi.fit(X_multi, y_multi)
            y_pred_multi = model_multi.predict(X_multi)

            coef_deter_multi = model_multi.score(X_multi, y_multi)
            coef_correl_multi = np.sqrt(coef_deter_multi)
            
            coeficientes = dict(zip(varIndependiente, model_multi.coef_))
            intercepto = model_multi.intercept_

            ecuacion_multi = " + ".join([f"{coef:.4f}·{var}" for var, coef in coeficientes.items()])
            ecuacion_multi = f"y = {ecuacion_multi} + {intercepto:.4f}"

            # Guardar la gráfica y los datos en session_state
            st.session_state["graficas_guardadas_multiple"].append({
                "var_dependiente": varDependiente,
                "var_independiente": varIndependiente,
                "ecuacion": ecuacion_multi,
                "r": coef_correl_multi,
                "r2": coef_deter_multi,
                "coeficientes": coeficientes,
                "intercepto": intercepto,
                "id": len(st.session_state["graficas_guardadas_multiple"])
            })


    # ---------------------------
    # Mostrar las gráficas guardadas de la múltiple
    if st.session_state["graficas_guardadas_multiple"]:
        st.subheader("Regresión Lineal Múltiple")

        for grafica in st.session_state["graficas_guardadas_multiple"]:
            st.subheader(' - '.join(grafica['var_independiente']))
            st.write(f"Variables independientes: {', '.join(grafica['var_independiente'])}")
            st.write(f"Variable dependiente: {grafica['var_dependiente']}")
            st.write(f"Ecuación del modelo: {grafica['ecuacion']}")
            st.write(f"Coeficiente de correlación (r): {grafica['r']:.4f}")
            st.write(f"Coeficiente de determinación (R²): {grafica['r2']:.4f}")

            # -------------------
            # Generar una gráfica por cada independiente
            for independiente in grafica['var_independiente']:
                X_plot = df[[independiente]].values
                y_plot = df[grafica['var_dependiente']].values

                model = LinearRegression()
                model.fit(X_plot, y_plot)
                y_pred_plot = model.predict(X_plot)

                fig = px.scatter(df, 
                                x=independiente, 
                                y=grafica['var_dependiente'],
                                title=f"{grafica['var_dependiente']} vs {independiente}")
                fig.add_scatter(x=df[independiente], y=y_pred_plot,
                                mode='lines', name='Línea de regresión', line=dict(color='red'))
                st.plotly_chart(fig, use_container_width=True, key=f"{grafica['id']}_{independiente}")


            # -------------------
            # Botón para eliminar cada gráfica
            if st.button(f"Eliminar gráficas", key=f"del_multiple_{grafica['id']}"):
                st.session_state["graficas_guardadas_multiple"] = [
                    g for g in st.session_state["graficas_guardadas_multiple"] if g["id"] != grafica["id"]
                ]
                st.rerun()
                break
    
    # Verifica si hay gráficas generadas o si está activado el heatmap
    if not st.session_state.get("graficas_guardadas") and not st.session_state.get("graficas_guardadas_multiple"):
        st.markdown("""
            <div style="background-color:#90E0EF; padding:15px; border-radius:10px; color:#003049;">
                <h4>Regresión lineal y análisis de correlación</h4>
                <p>La <strong>regresión lineal</strong> es una técnica estadística que permite analizar la relación entre una variable dependiente y una o más variables independientes.</p>
                <p>En la <strong>regresión lineal simple</strong>, se estudia la relación entre una sola variable independiente y la variable dependiente. Por otro lado, la <strong>regresión lineal múltiple</strong> permite evaluar cómo varias variables independientes, en conjunto, influyen sobre la variable dependiente.</p>
                <p>El <strong>mapa de calor (heatmap)</strong> es una herramienta visual que muestra el grado de correlación entre las variables numéricas del conjunto de datos, ayudando a identificar relaciones fuertes, débiles o inexistentes entre ellas.</p>
                <p>Para comenzar a generar las gráficas, selecciona las variables que deseas analizar y presiona el botón <strong>"Generar gráfica"</strong> para regresión simple o múltiple, o activa la opción <strong>"Mostrar mapa de calor"</strong> si deseas visualizar la matriz de correlación.</p>
            </div>
        """, unsafe_allow_html=True)

elif tipo_vista == "Análisis logistico":
    st.header("Regresión logistica")

    # Copia del dataframe
    df_log = df.copy()

    
    df_log['host_is_superhost'] = df_log['host_is_superhost'].map({'t': 1, 'f': 0})
    df_log['host_identity_verified'] = df_log['host_identity_verified'].map({'t': 1, 'f': 0})
    df_log['instant_bookable'] = df_log['instant_bookable'].map({'t': 1, 'f': 0})
    df_log['has_availability'] = df_log['has_availability'].map({'t': 1, 'f': 0})

 
    variables_independientes = [
        "price", "review_scores_rating", "number_of_reviews", "availability_365",
        "host_response_rate", "host_acceptance_rate", "host_listings_count",
        "host_total_listings_count", "accommodates", "bedrooms", "beds", "bathrooms",
        "availability_30", "availability_90", "review_scores_accuracy", 
        "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication",
        "review_scores_location", "review_scores_value"
    ]

 
    variables_dependientes = [
        "host_is_superhost", "host_identity_verified", "instant_bookable", "has_availability"
    ]
    # Selección de variables
    variable_dependiente = st.sidebar.selectbox("Variable dependiente (dicotomica)", variables_dependientes)
    variables_independientes = st.sidebar.multiselect("Variables independientes", options=variables_independientes)
    generar_log = st.sidebar.button("Generar modelo")
    # ------------------------------
    if generar_log and variables_independientes:

        
        st.subheader(f"Variables utilizadas")
        st.write(f"**Variables Independientes**: {', '.join(variables_independientes)}")
        st.write(f"**Variable Dependiente**: {variable_dependiente}") 

        df_copy = df_log.copy()
        X = df_copy[variables_independientes]
        y = df_copy[variable_dependiente]

        # Entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        # Escalado
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        # Modelo
        modelo = LogisticRegression()
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        # Métricas
        precision = precision_score(y_test, y_pred, average='binary', zero_division=1)
        exactitud = accuracy_score(y_test, y_pred)
        sensibilidad = recall_score(y_test, y_pred, average='binary')

        # Mostrar métricas
        st.subheader("Métricas del Modelo")
        st.write(f"**Precisión**: {round(precision, 4)}")
        st.write(f"**Exactitud**: {round(exactitud, 4)}")
        st.write(f"**Sensibilidad**: {round(sensibilidad, 4)}")

        # ------------------------------
        # Matriz de Confusión
        cm = confusion_matrix(y_test, y_pred)
        st.subheader("Matriz de Confusión")

        # Crear el gráfico
        fig, ax = plt.subplots(figsize=(6, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Sí'], yticklabels=['No', 'Sí'])
        plt.xlabel('Predicción')
        plt.ylabel('Real')
        plt.title(f'Matriz de Confusión - {variable_dependiente}')
        st.pyplot(fig)

    # ------------------------------
    # Descripción de la regresión logística
    else:
        st.markdown("""
            <div style="background-color:#90E0EF; padding:15px; border-radius:10px; color:#003049;">
                <h4>Regresión Logística</h4>
                <p>La <strong>regresión logística</strong> es una técnica estadística que permite analizar la relación entre una variable dependiente binaria
                (con solo dos posibles resultados, como "sí" o "no") y una o varias variables independientes.</p>
                <p>Este tipo de análisis es ideal para <strong>predecir probabilidades</strong> o clasificar resultados en dos grupos, por ejemplo:
                saber si un host es <strong>superhost</strong> o no, si un alojamiento tiene <strong>disponibilidad</strong> o no,
                según características como precio, número de reseñas, calificaciones, etc.</p>
                <p>Para generar el modelo, selecciona la variable dependiente (que debe ser binaria) y las variables independientes,
                y luego presiona el botón <strong>"Generar modelo"</strong>. Los resultados aparecerán en la tabla de abajo con las métricas de precisión, sensibilidad y exactitud.</p>
            </div>
        """, unsafe_allow_html=True)


            #streamlit run app.py


Overwriting app.py
